# Day 37

Today I continued my work from yesterday. My main goal was to quantify the percentage of top finishes which I've defined as a player drafted in the first round of a X-team league who ended up finishing in the top X in fantasy points. So if a player was drafted in the first round of 10-team leagues for 4 seasons, and finished top 10 in fantasy points 3 out of 4 seasons, that player would have a top finish 75% of the time.

I decided to find the number of times a player was drafted in the first round by using a correlated subquery.

In [2]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

## Query the Data

In [7]:
query = """
WITH season_summary AS (
    SELECT
        player_id,
        player_display_name AS name,
        season,
        position,
        recent_team AS team,
        ROUND(SUM(fantasy_points),2) AS tot_pts,
        ROUND(SUM(fantasy_points_ppr),2) AS tot_pts_ppr
    FROM weekly
    WHERE season_type = 'REG' 
        AND player_display_name IS NOT NULL
    GROUP BY player_id, player_display_name, season),
rankings AS (
    SELECT
        *,
        RANK() OVER(PARTITION BY season ORDER BY tot_pts DESC) AS r_pts,
        RANK() OVER(PARTITION BY season ORDER BY tot_pts_ppr DESC) AS r_pts_ppr
    FROM season_summary), 
joined AS (   
    SELECT *
    FROM adp
    LEFT JOIN rankings
        ON rankings.name = adp.name
        AND rankings.season = adp.season
        AND rankings.position = adp.position
        -- AND rankings.team = adp.team -- this can cause missing values
    ), 
standard_top_performers AS (
    SELECT
        season,
        teams,
        scoring,
        name,
        position,
        team,
        adp_formatted,
        round,
        pick,
        tot_pts,
        -- tot_pts_ppr,
        r_pts
        -- r_pts_ppr
    FROM joined
    -- Only want to look at standard
    WHERE scoring = 'standard'
        AND (
            (teams = 10
                AND round = 1
                AND r_pts BETWEEN 1 AND 10)
            OR (teams = 12
                AND round = 1
                AND r_pts BETWEEN 1 AND 12)
        )),
ppr_top_performers AS (
    SELECT
        season,
        teams,
        scoring,
        name,
        position,
        team,
        adp_formatted,
        round,
        pick,
        -- tot_pts,
        tot_pts_ppr,
        -- r_pts
        r_pts_ppr
    FROM joined
    -- Only want to look at ppr
    WHERE scoring = 'ppr'
        AND (
            (teams = 10
                AND round = 1
                AND r_pts_ppr BETWEEN 1 AND 10)
            OR (teams = 12
                AND round = 1 
                AND r_pts_ppr BETWEEN 1 AND 12) 
        )), 
summary_table AS (
    SELECT 
        scoring, 
        teams, 
        name AS player_name, 
        position, COUNT(*) AS top_finishes,
        -- Correlated subquery to get the number of seasons player was picked in the 1st round
        (SELECT COUNT(*)
        FROM joined 
        WHERE name = ppr_top_performers.name
            AND scoring = ppr_top_performers.scoring
            AND teams = ppr_top_performers.teams
            AND round = 1
        ) AS num_round_1
    FROM ppr_top_performers
    GROUP BY teams, name
    HAVING top_finishes > 1
    ORDER BY teams, top_finishes DESC
)
SELECT
    *,
    ROUND(1.0 * top_finishes / num_round_1, 2) AS pct_top_finishes
FROM summary_table
ORDER BY scoring, teams, pct_top_finishes DESC
"""

df = pd.read_sql(query, conn)
df

,scoring,teams,player_name,position,top_finishes,num_round_1,pct_top_finishes
0,ppr,10,Aaron Rodgers,QB,2,2,1.00
1,ppr,10,Antonio Brown,WR,3,4,0.75
2,ppr,10,Davante Adams,WR,2,3,0.67
3,ppr,10,Alvin Kamara,RB,2,4,0.50
4,ppr,10,LeSean McCoy,RB,2,5,0.40
5,ppr,10,Ezekiel Elliott,RB,2,6,0.33
6,ppr,12,Aaron Rodgers,QB,2,2,1.00
7,ppr,12,Antonio Brown,WR,3,4,0.75
8,ppr,12,Davante Adams,WR,2,3,0.67
9,ppr,12,Arian Foster,RB,2,3,0.67


Some takeaways:
- Aaron Rodgers is the only QB to make it on this list and the only one who delivered 100% of the time when picked in the first round
- Ezekiel Elliott has been drafted in the first round the most but only finishes at the top 33-50% of the time
- I'd argue Antonio Brown has performed the best out of these players. Drafted in the first round 4 seasons and finished at the top 75% of the time. 